# Deploying Machine Learning Models

In the previous session we trained a model for predicting churn and evaluated it. Now let's deploy it

In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

## Data preparation

In [1]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [2]:
!wget $data -O data-week-3.csv 

--2023-10-17 15:52:12--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: 'data-week-3.csv'

data-week-3.csv     100%[===================>] 954.59K  --.-KB/s    in 0.007s  

2023-10-17 15:52:13 (135 MB/s) - 'data-week-3.csv' saved [977501/977501]



In [5]:
df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   seniorcitizen     7043 non-null   int64  
 3   partner           7043 non-null   object 
 4   dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   phoneservice      7043 non-null   object 
 7   multiplelines     7043 non-null   object 
 8   internetservice   7043 non-null   object 
 9   onlinesecurity    7043 non-null   object 
 10  onlinebackup      7043 non-null   object 
 11  deviceprotection  7043 non-null   object 
 12  techsupport       7043 non-null   object 
 13  streamingtv       7043 non-null   object 
 14  streamingmovies   7043 non-null   object 
 15  contract          7043 non-null   object 
 16  paperlessbilling  7043 non-null   object 


In [6]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [7]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

## k-fold Cross-Validation

This task was previously performed in week 4 with multiple values of C.

In [16]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)

    return dv, model

In [17]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [18]:
C = 1.0
n_splits = 5

In [19]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.840 +- 0.008


In [20]:
scores

[0.8420610561185641,
 0.8455730201297841,
 0.830744531406783,
 0.8301724275756219,
 0.851750023532365]

## Training model

Training best model with C = 1.0

In [25]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)

y_test = df_test.churn.values

y_pred = predict(df_test, dv,model)

auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

## Save the model

In [26]:
import pickle

In [27]:
output_file = f'model_C={C}.bin'

In [28]:
output_file

'model_C=1.0.bin'

In [29]:
f_out = open(output_file, 'wb')
pickle.dump((dv, model), f_out)
f_out.close()

In [30]:
!ls -lh *.bin

-rw-r--r-- 1 studio-lab-user users 2.5K Oct 17 17:41 'model_C=1.0.bin'


In [31]:
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

## Load the model

In [32]:
import pickle

In [33]:
input_file = 'model_C=1.0.bin'

In [34]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [35]:
model

LogisticRegression(max_iter=1000)

In [36]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [37]:
X = dv.transform([customer])

In [38]:
y_pred = model.predict_proba(X)[0, 1]

In [39]:
print('input:', customer)
print('output:', y_pred)

input: {'gender': 'female', 'seniorcitizen': 0, 'partner': 'yes', 'dependents': 'no', 'phoneservice': 'no', 'multiplelines': 'no_phone_service', 'internetservice': 'dsl', 'onlinesecurity': 'no', 'onlinebackup': 'yes', 'deviceprotection': 'no', 'techsupport': 'no', 'streamingtv': 'no', 'streamingmovies': 'no', 'contract': 'month-to-month', 'paperlessbilling': 'yes', 'paymentmethod': 'electronic_check', 'tenure': 1, 'monthlycharges': 29.85, 'totalcharges': 29.85}
output: 0.636358415272714


## Train script

Save notebook as a Python script.  
Edit the script and save it as ```train.py```.  
Execute train.py to save the trained moldel as ```model_C=1.0.bin```

## Virtual environment

Install ```pipenv``` to create a virtual environment:
```bash
$ pip install pipenv
```
Install required packages. It will create Pipfile and Pipfile.lock files.
```bash
$ pipenv install numpy scikit-learn==0.24.2 flask gunicorn
```

### Optional

Output virtualenv information.
```bash
$ pipenv --venv
```
List installed packages:
```bash
$ pipenv run pip list
```
Remove virtual environment (on current folder):
```bash
$ pipenv --rm
```

## Making requests to a Flask server

Run the Flask app ```predict.py``` on the ```pipenv``` environment:

```bash
$ pipvenv shell
$ python predict.py
```

Make a request to the app:

In [9]:
import requests

In [10]:
url = 'http://localhost:9696/predict'

In [11]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'two_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [12]:
response = requests.post(url, json=customer).json()

In [13]:
response

{'churn': True, 'churn_probability': 0.5133820686192991}

In [14]:
if response['churn']:
    print('sending email to', 'asdx-123d')

sending email to asdx-123d


## Making requests with gunicorn

Run ```predict.py``` on the pipenv environment:

```bash
$ pipvenv shell
$ gunicorn --bind 0.0.0.0:9696 predict:app
```

Run ```predict-test.py``` to make a request:

```bash
$ python predict-test.py
```

## Making request using a Docker container

1. Create a Dockerfile.
2. Build a Docker image:
```bash
$ docker build -t zoomcamp-test .
```
3. Run a Docker container:
```bash
$ docker run -it -rm -p 9696:9696 zoomcamp-test:latest
```
4. Make an inference using the model served by the container by running the following script (on the ml-zoompcamp env which has the ```requests``` package installed):
```bash
$ python predict_test.py
```
5. Optional: Explore the container by opening a Bash terminal:
```bash
$ docker run -it --rm --entrypoint=bash zoomcamp-test:latest
```